In [1]:
from langdetect import detect, detect_langs
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
import math
from math import exp
import csv
%matplotlib inline

E:\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## word2vec score column

> #### 1. processiong_question_text Dataframe 불러오기

In [2]:
final_df = pd.read_csv('./train_final5.csv')
final_df.head(10)

,Unnamed: 0,qid,origin_question_text,remove_string_special,lemmatized,target
0,0,00002165364db923c7e6,How did Quebec nationalists see their province...,How did Quebec nationalists see their province...,How do Quebec nationalist see their province a...,0
1,1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",Do you have an adopted dog how would you encou...,Doe you have an adopt dog how would you encour...,0
2,2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,Why does velocity affect time Does velocity af...,Why do velocity affect time Does velocity affe...,0
3,3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,How did Otto von Guericke used the Magdeburg h...,How do Otto von Gerick use the Magruder hemisp...,0
4,4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,Can convert montra helicon to mountain bike by...,Can convert contra Helicon to mountain bike by...,0
5,5,00004f9a462a357c33be,"Is Gaza slowly becoming Auschwitz, Dachau or T...",Is Gaza slowly becoming Auschwitz Dachau or Tr...,Is Gaza slowly become Auschwitz Dachau or Treb...,0
6,6,00005059a06ee19e11ad,Why does Quora automatically ban conservative ...,Why does Quora automatically ban conservative ...,Why do Quorate automatically ban conservative ...,0
7,7,0000559f875832745e2e,Is it crazy if I wash or wipe my groceries off...,Is it crazy if wash or wipe my groceries off G...,Is it crazy if wash or wipe my grocery off Ger...,0
8,8,00005bd3426b2d0c8305,"Is there such a thing as dressing moderately, ...",Is there such thing as dressing moderately and...,Is there such thing a dress moderately and if ...,0
9,9,00006e6928c5df60eacb,Is it just me or have you ever been in this ph...,Is it just me or have you ever been in this ph...,Is it just me or have you ever be in this phas...,0


In [3]:
final_df.loc[30]['lemmatized']

'Which baby be more sweeter to their parent Dark skin baby or light skin baby'

> #### 2. 전체 question_text에 대해 tokenizedlist 만들기 

In [48]:
train_target = final_df["lemmatized"]

In [49]:
train_target = train_target.apply(lambda x: str(x))
train_corpus = list(train_target.apply(lambda x: word_tokenize(x)))

In [55]:
train_corpus[30]

['Which',
 'baby',
 'be',
 'more',
 'sweeter',
 'to',
 'their',
 'parent',
 'Dark',
 'skin',
 'baby',
 'or',
 'light',
 'skin',
 'baby']

> #### 3. word2vec의 대상이 될 target=1 단어들을 100차원의 벡터행렬로 만들기

In [7]:
embedding_model = Word2Vec(train_corpus, size=100, window = 2, min_count=3, workers=2, iter=100, sg=1)

- 임베딩 후 단어의 종류

In [8]:
embedding_model.wv.index2word

['be',
 'the',
 'What',
 'to',
 'in',
 'of',
 'do',
 'How',
 'and',
 'for',
 'you',
 'it',
 'Why',
 'have',
 'can',
 'Is',
 'that',
 'my',
 'with',
 'on',
 'or',
 'get',
 'from',
 'an',
 'your',
 'if',
 'best',
 'people',
 'a',
 'some',
 'there',
 'Can',
 'would',
 'like',
 'not',
 'when',
 'should',
 'Which',
 'make',
 'at',
 'about',
 'use',
 'by',
 'they',
 'Doe',
 'any',
 'good',
 'we',
 'so',
 'what',
 'me',
 'will',
 'their',
 'one',
 'If',
 'India',
 'who',
 'think',
 'Are',
 'after',
 'most',
 'year',
 'way',
 'take',
 'work',
 'how',
 'time',
 'but',
 'this',
 'know',
 'more',
 'go',
 'all',
 'many',
 'between',
 'than',
 'want',
 'much',
 'Does',
 'life',
 'other',
 'someone',
 'Who',
 'well',
 'become',
 'out',
 'book',
 'feel',
 'start',
 'Where',
 'he',
 'country',
 'thing',
 'up',
 'dont',
 'why',
 'job',
 'which',
 'find',
 'them',
 'give',
 'ever',
 'say',
 'learn',
 'woman',
 'person',
 'his',
 'Should',
 'without',
 'Im',
 'need',
 'just',
 'US',
 'into',
 'student',


In [9]:
len(embedding_model.wv.index2word)

67088

In [31]:
embedding_model.

- 찾고자 하는 word의 index 찾기

In [42]:
cnt = 0
weight_word_list = ['white','muslims','americans','black','girls','indians', 'hate',
 'chinese','muslim','american','america',
 'jews','gay','questions','man','fuck',
 'racist','god','china','christians',
 'alabama','woman','since','hindus',
 'islam','stupid','pakistan','north',
 'israel','kill','guys','states', 'south', 'religion',
 'never','atheists','gun','war',
 'rape','tits']

weight_word_index_list = []

for word in embedding_model.wv.index2word:
    for weight_word in weight_word_list:
        if word == weight_word :
            weight_word_index_list.append(cnt)
    cnt +=1

In [43]:
weight_word_index_list

[104,
 247,
 249,
 251,
 258,
 272,
 310,
 348,
 430,
 476,
 515,
 581,
 823,
 969,
 1249,
 1260,
 1931,
 2174,
 2494,
 12970,
 26385,
 39636,
 48091,
 57179]

- embedding_model test

In [53]:
print(embedding_model.most_similar(positive=["black"], topn=10))

[('white', 0.9206088185310364), ('brown', 0.691301703453064), ('Black', 0.6657844185829163), ('African', 0.6525812745094299), ('Hispanic', 0.6395251154899597), ('yellow', 0.6385411024093628), ('nonwhite', 0.6384801268577576), ('red', 0.6357205510139465), ('blue', 0.6329607963562012), ('White', 0.6251962184906006)]


In [45]:
word2vec_vec = embedding_model.wv.syn0

In [46]:
word2vec_vec.shape

(67088, 100)

- 67088개의 단어가 100차원의 벡터로 표현되었다.

> #### 4. 단어사이의 유클리드 거리를 계산한다.

In [15]:
def dot(v, w):  # Dot product
    return sum(v_i * w_i for v_i, w_i in zip(v,w))

In [16]:
def sum_of_square(v):
    return dot(v, v)

In [17]:
def vector_substract(v, w):
    return [v_i - w_i for v_i, w_i in zip(v, w)]

In [18]:
def squared_distance(v, w):
    return sum_of_square(vector_substract(v,w))

In [19]:
def distance(v, w):
    return math.sqrt(squared_distance(v, w))

- ex) 0번째 단어와 1번째 단어의 거리 계산

In [54]:
distance(word2vec_vec[1],word2vec_vec[3])

2.78799937717712

- ex) 0번째 단어와 0번째 단어의 거리 계산

In [21]:
distance(word2vec_vec[0],word2vec_vec[0])

0.0

- one-hot-encoding을 위하여 target=1 문장들의 모든 단어를 추출한다.

In [22]:
embed_word_list = embedding_model.wv.index2word

In [23]:
vect = CountVectorizer(vocabulary=embed_word_list)

In [ ]:
cnt_matrix = vect.transform(train_target).toarray()

- 유클리드거리를 구하는 함수

In [126]:
def make_distance_matrix(word2vec_vec):
    word2vec_matrix = np.zeros((len(weight_word_index_list),len(word2vec_vec)))
    
    for i,word_index in enumerate(weight_word_index_list):
        for j in range(len(word2vec_vec)):
            word2vec_matrix[i][j] = distance(word2vec_vec[word_index],word2vec_vec[j])
    
    return word2vec_matrix

In [127]:
distance_matrix = make_distance_matrix(word2vec_vec)
distance_matrix

array([[2.39106165, 2.75639085, 2.70771031, ..., 3.41620744, 4.13303557,
        3.73832753],
       [2.72570042, 2.81035341, 2.91557508, ..., 3.81623958, 4.42830025,
        3.84780673],
       [2.52191595, 2.59128166, 2.68416592, ..., 3.86250359, 4.01318791,
        3.79559784],
       ...,
       [4.06341148, 4.07055763, 4.3359327 , ..., 4.73869355, 4.56191896,
        4.51396159],
       [5.41018808, 5.36973043, 5.59040817, ..., 5.86029929, 5.62981002,
        5.6002294 ],
       [6.07205903, 6.05499453, 6.10803009, ..., 6.37683217, 6.89669377,
        6.10648584]])

In [128]:
distance_matrix.shape

(42, 16075)

> #### 5. 가중치 행렬을 구한다.

In [115]:
def make_weight_matrix(distance_matrix):
    word_weight_matrix = np.zeros((len(distance_matrix), len(distance_matrix[0])))
    
    std = np.std(word2vec_vec) 
    for i in range(len(distance_matrix)):
        for j in range(len(distance_matrix[0])):
            word_weight_matrix[i][j] = exp((-distance_matrix[i][j]**2)/(2*(std**2)))
    return word_weight_matrix

In [116]:
word_weight_matrix = make_weight_matrix(word2vec_matrix)
word_weight_matrix

array([[1.84527705e-06, 2.39901652e-08, 4.43413148e-08, ...,
        1.97382744e-12, 7.37790496e-18, 9.63679686e-15],
       [5.26268401e-09, 2.07291099e-09, 3.97363816e-10, ...,
        1.29770116e-15, 5.46254381e-20, 1.38316302e-16],
       [2.02094791e-09, 3.03543317e-10, 1.09486012e-10, ...,
        1.02821552e-16, 1.50217582e-20, 3.52460506e-14],
       ...,
       [1.48516071e-13, 3.37424419e-14, 5.90359203e-15, ...,
        4.94000702e-20, 1.63080610e-26, 9.73019919e-21],
       [1.62225551e-15, 1.95737252e-16, 8.92709925e-18, ...,
        5.02157656e-15, 1.20330719e-24, 1.65448003e-24],
       [4.94829132e-17, 8.74543166e-21, 6.10953047e-18, ...,
        4.92233270e-20, 5.00880851e-24, 6.52944370e-22]])

> #### 6. 각 문장의 스코어를 구한다.

In [117]:
def make_scored_matrix(word_weight_matrix,cnt_matrix):
    scored_matrix = []
    for i in cnt_matrix:
        scored_matrix.append(np.dot(word_weight_matrix,i))
    return scored_matrix

In [118]:
scored_matrix = make_scored_matrix(word_weight_matrix,cnt_matrix[:100])
scored_matrix

[array([6.83762584e-08, 2.47101308e-09, 4.13449695e-10, 3.64733055e-18,
        1.06299040e-14, 3.96918361e-14, 2.04845356e-16, 6.11830034e-18]),
 array([1.39091758e-06, 2.07040854e-10, 1.46253540e-10, 7.14015161e-17,
        7.03502041e-15, 6.14825700e-15, 3.38838241e-16, 1.09711212e-16]),
 array([8.86827502e-08, 7.94728209e-10, 2.18972369e-10, 7.22988119e-18,
        2.47638390e-16, 1.18267835e-14, 1.86466743e-17, 1.22194412e-17]),
 array([6.83314800e-08, 2.47027489e-09, 4.13029384e-10, 3.64733695e-18,
        8.56225519e-15, 3.96461184e-14, 2.04664353e-16, 6.25324391e-18]),
 array([2.10495376e-07, 1.36559508e-09, 4.59731908e-10, 5.73229935e-18,
        7.94961105e-14, 3.74223278e-13, 1.18067235e-16, 6.04297592e-18]),
 array([3.87709581e-24, 6.34449970e-25, 1.06958877e-26, 2.12200790e-35,
        7.99289977e-29, 1.25621672e-27, 2.26524299e-29, 4.00735366e-32]),
 array([1.33031040e-07, 1.19285443e-09, 3.28602758e-10, 1.08448339e-17,
        3.73205137e-16, 1.79052331e-14, 2.77607914e-